# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [224]:
import requests
import pandas as pd
import os

# Retrieve the DataFrame from the IPython store
%store -r df_bike
#df_bikes = pd.read_csv("city_bikes_ll_2.csv") # reading locations for bike stations (from previous task) // was checking alternative 
api_key = os.environ["location"]

headers = {"Accept": "application/json"} # Create dictionary for headers

headers['Authorization'] = api_key # Add key with our API KEY

fqr_restaurant_data = [] # here will store data for restaurants
i=0
# Loop through each bike station in df_bikes
for index, station in df_bikes.iterrows():
    i+=1
    latitude = station["Latitude"]
    longitude = station["Longitude"]
    station_id=station["StationID"] # will need in order to join together data from city_bikes,yelp and foursquare
    print(latitude,longitude,stationid)
    if i>100:
        break
    # Query Foursquare API for restaurants
    foursquare_url = "https://api.foursquare.com/v3/places/search"
    foursquare_params = {
        "ll": f"{latitude},{longitude}",
        "radius": 1000,  # Define the radius to search for venues around the bike station (in meters)
        "query": "restaurant",
        "limit":10 # limiting number of bussines results to 10
     }
    foursquare_response = requests.get(foursquare_url, headers=headers,params=foursquare_params)
    #print(foursquare_response)
    if foursquare_response.status_code == 200:
        foursquare_data = foursquare_response.json()
        #print(foursquare_data)
        for restaurant_fqr in foursquare_data["results"]:
             # Add the station ID to the restaurant data
            restaurant_fqr["station_id"] = station_id
            restaurant_fqr["Rating_fqr"]=0
        fqr_restaurant_data.extend(foursquare_data["results"])
    else:
        print(f"Failed to fetch Foursquare data for station {index}.")

43.665269 -79.319796 fb337bbed72e2be090071e199899b2be
43.67142 -79.445947 fb337bbed72e2be090071e199899b2be
43.666224 -79.317693 fb337bbed72e2be090071e199899b2be
43.653236 -79.376716 fb337bbed72e2be090071e199899b2be
43.663722 -79.380288 fb337bbed72e2be090071e199899b2be
43.657763 -79.389165 fb337bbed72e2be090071e199899b2be
43.656729 -79.382736 fb337bbed72e2be090071e199899b2be
43.664467 -79.414783 fb337bbed72e2be090071e199899b2be
43.675492 -79.388858 fb337bbed72e2be090071e199899b2be
43.674991 -79.396273 fb337bbed72e2be090071e199899b2be
43.671944 -79.387778 fb337bbed72e2be090071e199899b2be
43.662365 -79.351104 fb337bbed72e2be090071e199899b2be
43.646144 -79.377962 fb337bbed72e2be090071e199899b2be
43.663993 -79.358534 fb337bbed72e2be090071e199899b2be
43.657266 -79.374756 fb337bbed72e2be090071e199899b2be
43.6672144 -79.3452697 fb337bbed72e2be090071e199899b2be
43.673939 -79.441159 fb337bbed72e2be090071e199899b2be
43.657991 -79.340075 fb337bbed72e2be090071e199899b2be
43.66745 -79.329408 fb337bb

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone,station_id
0,4ad9ebdcf964a520e61b21e3,"[{'id': 13049, 'name': 'Diner', 'icon': {'pref...",[],129,"{'main': {'latitude': 43.665121, 'longitude': ...",/v3/places/4ad9ebdcf964a520e61b21e3,"{'address': '1483 Queen St E', 'country': 'CA'...",Chick-N-Joy,{},America/Toronto,fb337bbed72e2be090071e199899b2be
1,4b65b9f1f964a520fefb2ae3,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",[{'id': 'ab4a3530-d68a-012e-5619-003048cad9da'...,62,"{'main': {'latitude': 43.665065, 'longitude': ...",/v3/places/4b65b9f1f964a520fefb2ae3,"{'address': '31 Woodward Ave', 'address_extend...",McDonald's,{},America/Toronto,fb337bbed72e2be090071e199899b2be
2,4fd52f42e4b0b916eb02ab1b,"[{'id': 13064, 'name': 'Pizzeria', 'icon': {'p...",[],300,"{'drop_off': {'latitude': 43.666686, 'longitud...",/v3/places/4fd52f42e4b0b916eb02ab1b,"{'address': '6 Coxwell Ave', 'country': 'CA', ...",O Sushi,{},America/Toronto,fb337bbed72e2be090071e199899b2be
3,4afec89ef964a520623022e3,"[{'id': 13049, 'name': 'Diner', 'icon': {'pref...",[],265,"{'main': {'latitude': 43.666394, 'longitude': ...",/v3/places/4afec89ef964a520623022e3,"{'address': '1606 Queen St E', 'country': 'CA'...",Tulip Restaurant,{},America/Toronto,fb337bbed72e2be090071e199899b2be
4,4c1169396e5dc9b61b10b02d,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",[],377,"{'main': {'latitude': 43.666709, 'longitude': ...",/v3/places/4c1169396e5dc9b61b10b02d,"{'address': '1636 Queen St E', 'country': 'CA'...",Null,{},America/Toronto,fb337bbed72e2be090071e199899b2be


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [225]:

fqr_restaurant_df = pd.DataFrame(fqr_restaurant_data)

# Initialize lists to store the extracted data from foursquere
fqr_names = []
fqr_latitudes = []
fqr_longitudes = []
fqr_distances = []
fqr_station_ids=[]
fqr_ratings=[]

for index in range(len(fqr_restaurant_df)):
    fqr_longitude =fqr_restaurant_df['geocodes'].iloc[index]['main']['longitude']
    fqr_latitude =fqr_restaurant_df['geocodes'].iloc[index]['main']['latitude']
    fqr_name=fqr_restaurant_df.iloc[index]['name']
    fqr_distance=fqr_restaurant_df.iloc[index]['distance']
    fqr_station_id=fqr_restaurant_df.iloc[index]['station_id']
    fqr_rating=fqr_restaurant_df.iloc[index]['Rating_fqr']
    
    # Append the data to the corresponding lists
    fqr_names.append(fqr_name)
    fqr_latitudes.append(fqr_latitude)
    fqr_longitudes.append(fqr_longitude)
    fqr_distances.append(fqr_distance)
    fqr_station_ids.append(fqr_station_id)
    fqr_ratings.append(fqr_rating)
    
# creating new dataframe with selected columns
fqr_restaurant_df_selected=pd.DataFrame({
    'Name': fqr_names,
    'Latitude': fqr_latitudes,
    'Longitude': fqr_longitudes,
    'Distance': fqr_distances,
    'Rating':fqr_ratings,
    'Station_id':fqr_station_ids
})

fqr_restaurant_df_selected
%store fqr_restaurant_df_selected

Stored 'fqr_restaurant_df_selected' (DataFrame)


In [226]:
fqr_restaurant_df_selected

,Name,Latitude,Longitude,Distance,Rating,Station_id
0,Chick-N-Joy,43.665121,-79.321347,129,0,fb337bbed72e2be090071e199899b2be
1,McDonald's,43.665065,-79.319000,62,0,fb337bbed72e2be090071e199899b2be
2,O Sushi,43.666623,-79.316861,300,0,fb337bbed72e2be090071e199899b2be
3,Tulip Restaurant,43.666394,-79.316918,265,0,fb337bbed72e2be090071e199899b2be
4,Null,43.666709,-79.315659,377,0,fb337bbed72e2be090071e199899b2be
...,...,...,...,...,...,...
972,Bairrada Churrasqueira,43.649992,-79.434052,286,0,4c18c44e6f93e57321d327124945cb19
973,Le Baratin,43.649970,-79.435123,314,0,4c18c44e6f93e57321d327124945cb19
974,Immanishi Sando Bar,43.647041,-79.433719,86,0,4c18c44e6f93e57321d327124945cb19
975,Uncle Mikey's,43.649703,-79.434940,296,0,4c18c44e6f93e57321d327124945cb19


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [227]:
import requests
import pandas as pd

#df_bikes = pd.read_csv("city_bikes_ll_2.csv") // was checking alternative 
yelp_api_key = "vaSYL7ayPIeJ-5HCAlEgA7OBkZWCqDsBbEY99dHSIeVCz92tWUa2NQWasZsMIC3KMohvXF1HJoZd_bTAkRX-H3lS6IXQHF7UgJGOLWCxI6_pbHJVPmrXk7FJytLFZHYx"

yelp_restaurant_data = [] # restoaurants from yelp
i=0
# Loop through each bike station in df_bikes
   # FOR EACH LOCATION FROM BIKES WILL GET 20 RESTAURANT LOCATIONS
for index, station in df_bikes.iterrows():
    i+=1
    latitude = station["Latitude"]
    longitude = station["Longitude"]
    station_id=station["StationID"] # will need in order to join together data from city_bikes,yelp and foursquare
    print(latitude,longitude)
    if i>100:
        break
    
    yelp_url = "https://api.yelp.com/v3/businesses/search"
    yelp_headers = {
            "Authorization": f"Bearer {yelp_api_key}"
        }
    yelp_params = {
       "latitude": latitude,
       "longitude": longitude,
       "radius": 1000,
       "term": "Restaurants", 
        "limit":10 # limiting number of bussines results to 10
    }
    yelp_response = requests.get(yelp_url, headers=yelp_headers, params=yelp_params)
    if yelp_response.status_code == 200:
        yelp_data = yelp_response.json()
        for restaurant_yelp in yelp_data["businesses"]:
        # Add the station ID to the restaurant data
            restaurant_yelp["station_id"] = station_id
        yelp_restaurant_data.extend(yelp_data["businesses"])
    else:
        print(f"Failed to fetch Yelp data for station {index}.")


43.665269 -79.319796
43.67142 -79.445947
43.666224 -79.317693
43.653236 -79.376716
43.663722 -79.380288
43.657763 -79.389165
43.656729 -79.382736
43.664467 -79.414783
43.675492 -79.388858
43.674991 -79.396273
43.671944 -79.387778
43.662365 -79.351104
43.646144 -79.377962
43.663993 -79.358534
43.657266 -79.374756
43.6672144 -79.3452697
43.673939 -79.441159
43.657991 -79.340075
43.66745 -79.329408
43.672453 -79.338259
43.6592088 -79.3348396
43.689211 -79.35438
43.656927 -79.378497
43.658295 -79.372346
43.6940155 -79.2889842
43.6809164 -79.4229684
43.6910667 -79.39535
43.6975969 -79.39654
43.639832 -79.395954
43.667333 -79.399429
43.6480008 -79.383177
43.656518 -79.389099
43.687999 -79.38906
43.6563889 -79.45166666666667
43.639179 -79.399595
43.6616667 -79.40138888888889
43.66666667 -79.45833333333334
43.657192 -79.452559
43.6576 -79.4032
43.660694 -79.379052
43.636533 -79.395854
43.659777 -79.382767
43.643667 -79.380414
43.666233 -79.373327
43.640114 -79.393249
43.653766 -79.4016759
43.6

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [228]:

# Create DataFrames _POI data (yelp restaurants)
yelp_restaurant_data_df= pd.DataFrame(yelp_restaurant_data)
print("\nYelp Restaurants:")
yelp_restaurant_data_df

# Initialize lists to store the extracted data from yelp
yelp_names = []
yelp_latitudes = []
yelp_longitudes = []
yelp_distances = []
yelp_ratings=[]
yelp_station_ids=[]

for index in range(len(yelp_restaurant_data_df)):
    yelp_name=yelp_restaurant_data_df.iloc[index]['name']
    yelp_latitude=yelp_restaurant_data_df.iloc[index]['coordinates']['latitude']
    yelp_longitude=yelp_restaurant_data_df.iloc[index]['coordinates']['longitude']   
    yelp_distance=yelp_restaurant_data_df.iloc[index]['distance']
    yelp_rating=yelp_restaurant_data_df.iloc[index]['rating']
    yelp_station_id=yelp_restaurant_data_df.iloc[index]['station_id']
    
    yelp_names.append(yelp_name)
    yelp_latitudes.append(yelp_latitude)
    yelp_longitudes.append(yelp_longitude)
    yelp_distances.append(yelp_distance)
    yelp_ratings.append(yelp_rating)
    yelp_station_ids.append(yelp_station_id)
    
#creating new dataframe with selected/filtered columns
yelp_restaurant_df_selected=pd.DataFrame({
    'Name': yelp_names,
    'Latitude': yelp_latitudes,
    'Longitude': yelp_longitudes,
    'Distance': yelp_distances,
    'Rating':yelp_ratings,
    'Station_id':yelp_station_ids
})
yelp_restaurant_df_selected
%store yelp_restaurant_df_selected


Yelp Restaurants:
Stored 'yelp_restaurant_df_selected' (DataFrame)


In [229]:
yelp_restaurant_df_selected

,Name,Latitude,Longitude,Distance,Rating,Station_id
0,Jaclyn's,43.666270,-79.318020,172.375134,4.5,fb337bbed72e2be090071e199899b2be
1,Lake Inez,43.672340,-79.320640,794.661955,4.5,fb337bbed72e2be090071e199899b2be
2,Casa Di Giorgios,43.666850,-79.315180,408.715724,4.0,fb337bbed72e2be090071e199899b2be
3,Betty’s East,43.663455,-79.327992,689.778721,4.5,fb337bbed72e2be090071e199899b2be
4,Maru Japanese Bistro,43.664720,-79.324263,364.448505,4.0,fb337bbed72e2be090071e199899b2be
...,...,...,...,...,...,...
989,Antler Kitchen & Bar,43.649850,-79.430590,316.096178,4.5,4c18c44e6f93e57321d327124945cb19
990,Le Baratin,43.650070,-79.435100,332.138639,4.5,4c18c44e6f93e57321d327124945cb19
991,The Federal,43.649770,-79.430110,336.993951,4.0,4c18c44e6f93e57321d327124945cb19
992,Nuna Kitchen & Bar,43.641540,-79.431120,678.653300,4.5,4c18c44e6f93e57321d327124945cb19


Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp provide more complete and structured data compared with Foursquere. Important parameters such as: alias, is closed, categories (type of restaurant), ratings and reviews, phone numbers exist only in Yelp.
Besides Yelp's structure is more orginized and easy to work with.
Parsing from Yelp was faster 58secs vs 90 secs with limitations of 100 stations and 10 restaurants
Yelp return 994 rows, Foursquare:977 rows.

Get the top 10 restaurants according to their rating

In [232]:
yelp_restaurant_df_top10 = yelp_restaurant_df_selected.sort_values(by='Rating', ascending=False).head(10)

yelp_restaurant_df_top10

,Name,Latitude,Longitude,Distance,Rating,Station_id
835,Papamio's,43.671960,-79.419280,241.184451,5.0,37c32c09c594b83701b3b8f6e32db605
917,Masa Buka,43.686488,-79.347888,723.520019,5.0,a7ca61373064c6581461132f1ae050de
154,East End Vine,43.666996,-79.344665,55.437143,5.0,784016bbba7f7be920647e7f8ec00251
904,Papyrus,43.677090,-79.353240,418.200804,5.0,ddba9c631e11fbae48e2be60289f60c2
827,Morellina’s,43.667250,-79.419999,313.120208,5.0,0abdfc3177000757dcd71bd4c8b479ee
397,Gurume Sushi,43.661391,-79.380995,174.501781,5.0,b086f4ccca33b1b976b40abe69b78d42
825,Papamio's,43.671960,-79.419280,249.328387,5.0,0abdfc3177000757dcd71bd4c8b479ee
824,Maison T,43.670520,-79.426320,445.221433,5.0,0abdfc3177000757dcd71bd4c8b479ee
43,Som Tum Jinda,43.660199,-79.378211,425.921440,5.0,8f8af40d9388c8a3962559e8681d3db7
232,Som Tum Jinda,43.660199,-79.378211,517.132788,5.0,e565aef96d6d5fce5279ef6f24b9ce69


note: 43 and 232 both have the same locations, names and ratings, but different distance. 
    Need further investigation to understand the reason. Probably one to be dropped.
    